<h1 align=center><font size = 10><strong>Capstone Project - The Battle of Neighborhoods<strong></font></h1>
<h1 align=center><font size = 10>Munich vs. Berlin</font></h1>
<h1 align=center><font size = 4>by Philipp Hock<br>January 11, 2021</font></h1>

<a id='top'></a>

## Table of Contents

<font size = 4>

1. [Introduction/Business Problem](#item1) <br>
    1.1 Background<br>
    1.2 focus and compromise <br>
    1.3 target <br>
    1.4 define restaurant categories to evaluate mean frequency of occurence<br>
    
2. [***Munich:*** Data import and cleaning](#item2) <br>
    2.1 libary import<br>
    2.2 Munich Data import with BeautifulSoup<br>
    2.3 Folium Map Munich<br>
    2.4 Explore Venues with 4square<br>
    
3. [***Munich:*** Analyze Boroughs](#item3) <br>
    3.1 Frequency of occurance<br>
    3.2 Cluster Borough with k-means<br>
    3.3 Cluster Visualization<br>
    3.4 evaluate target criteria according to 'munich_restaurants'<br>
    
4. [***Berlin:*** Data import and cleaning](#item4) <br>
    4.1 libary import<br>
    4.2 Munich Data import with BeautifulSoup<br>
    4.3 Folium Map Munich<br>
    4.4 Explore Venues with 4square<br>
    
5. [***Berlin:*** Analyze Boroughs](#item5) <br>
    5.1 Frequency of occurance<br>
    5.2 Cluster Borough with k-means<br>
    5.3 Cluster Visualization<br>
    5.4 evaluate target criteria according to 'munich_restaurants'<br>
    
6. [result and discussion](#item6) <br>
    6.1 Berlin <br>
    6.2 Munich <br>
    6.3 Trending venue
    <br>
<br>
</font>
</div>


<div class="alert alert-block alert-info">

<a id='item1'></a>

## 1. <a href='item1'>Introduction/Business Problem</a><br>
## 1.1 Problem description
Two of Germany's most wanted cities are Berlin and Munich. The first cities that come to everybody’s mind, when thinking about diversity and a place to be. However, if you think about having agony of choice to pic a restaurant, what would be your first thought? <br>
Exactly, 
- where can I find a restaurant? 
- which restaurants have the highest user ratings? 
- and of course, can I afford that? <br>

## 1.2 focus and compromise
In this analysis, we will focus on picking the right borough to find a suitable restaurant. If you do not use any Foursquare premium API requests, this will be the compromise to select your destination. 
Afterwards you can easily search the right restaurant via your mobile app and get the missing information.
<br>
We use free __*explore*__ requests to communicate with Foursquare database. As we decide for one borough, we will start one free premium request for __*trending*__ venues.

## 1.3 approach and target 
First we'll have a look at different venue categories offered by Foursquare. Second, we will create an array for relevant restaurants categories. So data of both cities are quite comparable as we start to evaluate amount of suitable restaurant categories. <br>In addition, we will use the _k_-means clustering algorithm to splitt restaurant into clusters according to venue categories by Foursquare. Finally, we will use the Folium library to visualize the neighborhoods in Berlin and Munich and their emerging clusters.<br>
Based on the results, my wife can decide whether she will have dinner in Munich or Berlin and to which borough I have to navigate her. <br> Anyway, she will only decide with her stomach. Rational evaluation of Foursquare data would exceed my competencies and is going to be outvoted.<br>
But as soon as we arrive, we will look for the places with the highest foot traffic. So let's do that and get the trending venues around.

## 1.5 define restaurant categories

Since we like to eat in different restaurants and experience new kitchens, we just need the number of restaurants for the first step, regardless of quality, price or user rating.
To be comparable, here's a list of venue categories, we would like to compare the cities:

In [2]:
restaurants = ['American Restaurant',
    'Asian Restaurant',
    'Argentinian Restaurant',
    'Fast Food Restaurant',
    'Bavarian Restaurant',
    'Chinese Restaurant',
    'Eastern European Restaurant',
    'English Restaurant',
    'Falafel Restaurant',
    'German Restaurant',
    'Greek Restaurant',
    'Indian Restaurant',
    'Israeli Restaurant',
    'Italian Restaurant',
    'Mediterranean Restaurant',
    'Mexican Restaurant',
    'Middle Eastern Restaurant',
    'Modern European Restaurant',
    'Restaurant',
    'Seafood Restaurant',
    'Sushi Restaurant',
    'Thai Restaurant',
    'Theme Restaurant',
    'Vegetarian / Vegan Restaurant',
    'Vietnamese Restaurant',
    'Beer Garden',
    'Bistro',
    'Burger Joint',
    'Burrito Place',
    'Creperie',
    'Fried Chicken Joint',
    'Pizza Place',
    'Steakhouse']

In [3]:
#munich_venues.to_csv('venue_categories_munich.csv', sep=';', decimal=',', index=True)
#berlin_venues.to_csv('venue_categories_berlin.csv', sep=';', decimal=',', index=True)

<a id='item2'></a>

## 2. Munich: Data import and cleaning
## 2.1 libary import

In [4]:
import pandas as pd
import numpy as np

!pip install BeautifulSoup4
from bs4 import BeautifulSoup
import requests

!pip install lxml
!pip install geopandas
!pip install geopy
import geopy as geo
import geopandas as gpd

import folium 
from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors

pd.options.mode.chained_assignment = None  # default='warn'
print('Done!')

     |████████████████████████████████| 122kB 5.4MB/s eta 0:00:01
     |████████████████████████████████| 5.5MB 8.5MB/s eta 0:00:01
     |████████████████████████████████| 972kB 6.4MB/s eta 0:00:01
     |████████████████████████████████| 1.0MB 20.0MB/s eta 0:00:01
     |████████████████████████████████| 14.8MB 38.8MB/s eta 0:00:01   |███████▋                        | 3.5MB 9.1MB/s eta 0:00:02     |██████████████████████████▉     | 12.4MB 38.8MB/s eta 0:00:01
     |████████████████████████████████| 6.5MB 8.6MB/s eta 0:00:01     |█████████████████████████████▉  | 6.0MB 8.6MB/s eta 0:00:01
  Found existing installation: pyproj 1.9.6
    Uninstalling pyproj-1.9.6:
      Successfully uninstalled pyproj-1.9.6
     |████████████████████████████████| 112kB 7.1MB/s eta 0:00:01
Done!


## 2.2 Munich Data import with beautifulSoup

In [5]:
url='http://www.places-in-germany.com/21179-places-within-a-radius-of-15km-around-muenchen.html'
req=requests.get(url)
soup=BeautifulSoup(req.text,"html.parser")
table = soup.find_all('table')
df=pd.read_html(str(table), header=0)[0]

In [6]:
df.head()

,Distance,Route,Postal code / Place,Population
0,3.4 km (2.1 miles),NaN,81675 Bogenhausen,-
1,4.3 km (2.7 miles),NaN,81671 Berg am Laim,-
2,5.1 km (3.2 miles),NaN,80796 Milbertshofen-Am Hart,-
3,6.7 km (4.2 miles),NaN,80634 Moosach,-
4,8.7 km (5.4 miles),NaN,85774 Unterföhring,7553


In [7]:
df.rename({'Postal code / Place':'Borough'},axis=1, inplace=True)
munich=df.Borough.str.split(" ",n=1,expand=True)
munich.rename({0:'postalcode', 1:'borough'}, axis=1, inplace=True)
munich.shape

(88, 2)

test for NaN values

In [8]:
if munich['borough'].isnull().sum() > 0:
    munich.dropna(axis=0, inplace=True)
    print('NaN values deleted')
else:
    print('no NaN values')

no NaN values


web adresse offers data with same postal code and borough name but different distances to our search request.<br> Therefore we will _drop duplicates()_

In [9]:
munich.drop_duplicates(keep='first', inplace=True)
munich.shape

(82, 2)

## 2.3 Folium Map Munich

In [10]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

lat=[]
lon=[]

geolocator = Nominatim(user_agent='Cousera')

for line, borough in munich.iterrows():
    
    try:
        adress= borough[0], ' München ', borough[1]
        location = geolocator.geocode(adress)
        #print(location)
        lat.append(location.latitude)
        lon.append(location.longitude)
    except:
        lat.append(np.nan)
        lon.append(np.nan)
    
munich['latitude']=lat
munich['longitude']=lon

In [11]:
munich.dropna(axis=0, inplace=True)
munich.shape

(67, 4)

In [12]:
munich

,postalcode,borough,latitude,longitude
0,81675,Bogenhausen,48.158487,11.636682
1,81671,Berg am Laim,48.123483,11.633451
2,80796,Milbertshofen-Am Hart,48.182385,11.575043
3,80634,Moosach,48.179895,11.510571
4,85774,Unterföhring,48.190718,11.644580
...,...,...,...,...
79,82110,Neugermering,48.127735,11.363372
82,85764,Mittenheim,48.263084,11.557159
83,82065,Baierbrunn,48.020477,11.486546
85,85630,Neukeferloh,48.096429,11.760195


#### add city center to dataset<br>
Folium Map shows, that there's no entry for the searched geo tag itself. It just returns positions around the search tag. So let's add it manually to the results of www.places-in-germany.com.  

In [13]:
df_city = pd.DataFrame({"postalcode":['80331', '80335'],"borough":['Zentrum Marienplatz', 'Hauptbahnhof'], "latitude":[48.137187, 48.140458], "longitude":[11.575501, 11.557766]})
df_city

,postalcode,borough,latitude,longitude
0,80331,Zentrum Marienplatz,48.137187,11.575501
1,80335,Hauptbahnhof,48.140458,11.557766


In [14]:
munich = munich.append(df_city, ignore_index=True)

In [15]:
address = 'munich'

geolocator = Nominatim(user_agent="Cousera")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('geo-coordinates of Munich {}, {}.'.format(latitude, longitude))



# Creating Folium Map
map_munich = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough in zip(munich['latitude'], munich['longitude'], munich['borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_munich)  
    
map_munich

geo-coordinates of Munich 48.1371079, 11.5753822.


## 2.4 Explore Venues with 4square

In [16]:
CLIENT_ID = 'MIXA2M5FTBG1IILFL5ATNOBSNUP4BU4J1BUOGIM03E1UJOBE' # your Foursquare ID
CLIENT_SECRET = 'ETQCTK31ZO4I0FVKYEEEHG2M0JGL0G4DCLQ1R1ECOOPHX33D' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
radius = 500
print("ID loaded")

ID loaded


In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['borough', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
munich_venues = getNearbyVenues(names=munich['borough'],
                                   latitudes=munich['latitude'],
                                   longitudes=munich['longitude']
                                  )

Bogenhausen
Berg am Laim
Milbertshofen-Am Hart
Moosach
Unterföhring
Dornach
Neuherberg
Pasing
Neubiberg
Pullach
Gronsdorf, Kreis München
Winning, Kreis München
Taufkirchen, Kreis München
Gräfelfing
Martinsried
Großhesselohe, Isartal
Pullach im Isartal
Ottobrunn
Westerham
Salmdorf, Kreis München
Unterhaching
Hochmutting
Oedenstockach
Steinkirchen
Solalinden
Taufkirchen
Aschheim
Dirnismaning bei München
Feldkirchen
Potzham, Kreis München
Gerblinghausen
Kreuzpullach
Laufzorn
Oberbiberg
Oberhaching bei München
Ödenpullach
Karlsfeld bei München
Putzbrunn
Ottendichl, Kreis München
Haar bei München
Planegg
Lustheim
Oberschleißheim
Hochbrück bei München
Badersfeld
Eglfing, Kreis München
Waldbrunn
Deisenhofen bei München
Ismaning
Buchenhain, Isartal
Hohenbrunn
Riemerling
Krailling
Garching
Garching bei München
Kirchheim bei München
Kirchheim
Grasbrunn
Frundsbergerhöhe
Kirchstockach, Kreis München
Rothschwaige
Harthaus
Neugermering
Mittenheim
Baierbrunn
Neukeferloh
Fischerhäuser
Zentrum Marienpl

In [19]:
print('4square provided', munich_venues.shape , 'venues')
munich_venues.head(5)

4square provided (697, 7) venues


,borough,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bogenhausen,48.158487,11.636682,Martinelli,48.155396,11.637115,Italian Restaurant
1,Bogenhausen,48.158487,11.636682,dm-drogerie markt,48.159468,11.643064,Drugstore
2,Bogenhausen,48.158487,11.636682,Pyrsos,48.154944,11.637708,Greek Restaurant
3,Bogenhausen,48.158487,11.636682,Rossmann,48.157856,11.641590,Drugstore
4,Bogenhausen,48.158487,11.636682,Bäckerei Wimmer,48.158156,11.641109,Bakery


In [20]:
print('There are {} uniques categories.'.format(len(munich_venues['Venue Category'].unique())))
munich_venues.groupby('Venue Category').Venue.count()

There are 163 uniques categories.


Venue Category
American Restaurant               1
Argentinian Restaurant            1
Art Museum                        1
Arts & Crafts Store               1
Asian Restaurant                 11
                                 ..
Trattoria/Osteria                 4
Vegetarian / Vegan Restaurant     1
Vietnamese Restaurant             5
Warehouse Store                   1
Wine Bar                          2
Name: Venue, Length: 163, dtype: int64

<a id='item3'></a>

## 3. Analyze Boroughs of Munich

In [21]:
munich_restaurants = munich_venues[munich_venues['Venue Category'].str.contains('|'.join(restaurants))]

In [22]:
munich_restaurants.head()

,borough,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bogenhausen,48.158487,11.636682,Martinelli,48.155396,11.637115,Italian Restaurant
2,Bogenhausen,48.158487,11.636682,Pyrsos,48.154944,11.637708,Greek Restaurant
20,Milbertshofen-Am Hart,48.182385,11.575043,Rabiang Thai,48.180777,11.571211,Thai Restaurant
21,Milbertshofen-Am Hart,48.182385,11.575043,Synantisis,48.179632,11.570367,Greek Restaurant
23,Milbertshofen-Am Hart,48.182385,11.575043,Wolfs-Burger,48.184465,11.571378,Burger Joint


## 3.1 Frequency of occurance

In [23]:
# one hot encoding
munich_onehot = pd.get_dummies(munich_restaurants[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
munich_onehot['borough'] = munich_restaurants['borough'] 

# move neighborhood column to the first column
fixed_columns = [munich_onehot.columns[-1]] + list(munich_onehot.columns[:-1])
munich_onehot = munich_onehot[fixed_columns]

munich_onehot.head()

,borough,American Restaurant,Argentinian Restaurant,Asian Restaurant,Bavarian Restaurant,Beer Garden,Bistro,Burger Joint,Burrito Place,Chinese Restaurant,...,Modern European Restaurant,Pizza Place,Restaurant,Seafood Restaurant,Steakhouse,Sushi Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Bogenhausen,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Bogenhausen,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20,Milbertshofen-Am Hart,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
21,Milbertshofen-Am Hart,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23,Milbertshofen-Am Hart,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


#### group rows by neighborhood and mean frequency of occurence of each category

In [24]:
print('Size of One Hot is ', munich_onehot.shape, '.')
munich_grouped = munich_onehot.groupby('borough').mean().reset_index()
munich_grouped.head()

Size of One Hot is  (183, 33) .


,borough,American Restaurant,Argentinian Restaurant,Asian Restaurant,Bavarian Restaurant,Beer Garden,Bistro,Burger Joint,Burrito Place,Chinese Restaurant,...,Modern European Restaurant,Pizza Place,Restaurant,Seafood Restaurant,Steakhouse,Sushi Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Aschheim,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Baierbrunn,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Bogenhausen,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Buchenhain, Isartal",0.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Deisenhofen bei München,0.0,0.0,0.500000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
print('confirm the new size:', munich_grouped.shape)

confirm the new size: (50, 33)


#### Let's print each neighborhood along with the top 5 most common venues

In [26]:
num_top_venues = 5

for hood in munich_grouped['borough']:
    print("----"+hood+"----")
    temp = munich_grouped[munich_grouped['borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Aschheim----
                           venue  freq
0               Greek Restaurant  0.33
1               Asian Restaurant  0.33
2             Italian Restaurant  0.33
3              Indian Restaurant  0.00
4  Vegetarian / Vegan Restaurant  0.00


----Baierbrunn----
                           venue  freq
0              German Restaurant   1.0
1            American Restaurant   0.0
2              Indian Restaurant   0.0
3  Vegetarian / Vegan Restaurant   0.0
4               Theme Restaurant   0.0


----Bogenhausen----
                           venue  freq
0               Greek Restaurant   0.5
1             Italian Restaurant   0.5
2         Argentinian Restaurant   0.0
3  Vegetarian / Vegan Restaurant   0.0
4               Theme Restaurant   0.0


----Buchenhain, Isartal----
                           venue  freq
0            Bavarian Restaurant   1.0
1            American Restaurant   0.0
2              Indian Restaurant   0.0
3  Vegetarian / Vegan Restaurant   0.0
4            

#### Let's put that into a _pandas_ dataframe

In [27]:
# First, let's write a function to sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

create the new dataframe and display top 5 venues

In [28]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['borough'] = munich_grouped['borough']

for ind in np.arange(munich_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(munich_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Aschheim,Greek Restaurant,Asian Restaurant,Italian Restaurant,Vietnamese Restaurant,Chinese Restaurant
1,Baierbrunn,German Restaurant,Vegetarian / Vegan Restaurant,Argentinian Restaurant,Asian Restaurant,Bavarian Restaurant
2,Bogenhausen,Greek Restaurant,Italian Restaurant,Vietnamese Restaurant,Chinese Restaurant,Falafel Restaurant
3,"Buchenhain, Isartal",Bavarian Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Argentinian Restaurant,Asian Restaurant
4,Deisenhofen bei München,Asian Restaurant,Greek Restaurant,Vietnamese Restaurant,Fried Chicken Joint,Argentinian Restaurant


## 3.2 Cluster Borough with k-means

In [29]:
# set number of clusters
kclusters = 10

munich_grouped_clustering = munich_grouped.drop('borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(munich_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50] 

array([0, 1, 0, 3, 0, 8, 2, 0, 7, 7, 7, 2, 2, 7, 2, 8, 7, 1, 1, 7, 1, 7,
       0, 5, 7, 8, 7, 3, 1, 0, 7, 6, 7, 7, 8, 2, 8, 8, 8, 0, 0, 4, 1, 1,
       2, 2, 0, 9, 7, 7], dtype=int32)

create dataframe

In [30]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

munich_merged = munich

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
# munich_merged = munich_merged.join(neighborhoods_venues_sorted.set_index('borough'), on='borough')

munich_merged = munich_merged.merge(neighborhoods_venues_sorted.set_index('borough'), on='borough')

munich_merged

,postalcode,borough,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,81675,Bogenhausen,48.158487,11.636682,0,Greek Restaurant,Italian Restaurant,Vietnamese Restaurant,Chinese Restaurant,Falafel Restaurant
1,80796,Milbertshofen-Am Hart,48.182385,11.575043,0,Greek Restaurant,German Restaurant,Thai Restaurant,Burger Joint,Asian Restaurant
2,80634,Moosach,48.179895,11.510571,7,German Restaurant,Italian Restaurant,Asian Restaurant,American Restaurant,Sushi Restaurant
3,85774,Unterföhring,48.190718,11.644580,2,Italian Restaurant,Vietnamese Restaurant,Fried Chicken Joint,Argentinian Restaurant,Asian Restaurant
4,85609,Dornach,48.153899,11.689900,8,German Restaurant,Thai Restaurant,Italian Restaurant,Burrito Place,Falafel Restaurant
5,81241,Pasing,48.147785,11.460701,7,Italian Restaurant,Vietnamese Restaurant,Fast Food Restaurant,German Restaurant,Theme Restaurant
6,85579,Neubiberg,48.076994,11.661514,8,German Restaurant,Italian Restaurant,Restaurant,Burrito Place,Falafel Restaurant
7,82049,Pullach,48.058962,11.521841,8,German Restaurant,Asian Restaurant,Italian Restaurant,Fried Chicken Joint,Argentinian Restaurant
8,85540,"Gronsdorf, Kreis München",48.108961,11.719957,7,Vietnamese Restaurant,Bistro,Italian Restaurant,Fried Chicken Joint,Argentinian Restaurant
9,82024,"Winning, Kreis München",48.052210,11.605970,7,Chinese Restaurant,Italian Restaurant,Doner Restaurant,Vietnamese Restaurant,Fried Chicken Joint


## 3.3 Cluster Visualization

In [31]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(munich_merged['latitude'], munich_merged['longitude'], munich_merged['borough'], munich_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 3.4 evaluate target criteria according to 'munich_restaurants'

In [32]:
munich_sum = pd.get_dummies(munich_restaurants[['Venue Category']], prefix="", prefix_sep="")

munich_sum['borough'] = munich_restaurants['borough']

#Fix Column Order
fixed_columns = [munich_sum.columns[-1]] + list(munich_sum.columns[:-1])
munich_sum = munich_sum[fixed_columns]

Sum amount of each venues out of target definition 'munich_restaurants'

In [82]:
munich_sum['amount of restaurants']=munich_sum.sum(axis=1)
munich_sum = munich_sum.groupby('borough').sum().reset_index()

keep only sum as criteria and add it to overview 'munich_merged'<br>
show only top5 boroughs according to amount of restaurants

In [84]:
munich_sum = munich_sum[['borough','amount of restaurants']]

munich_merged = pd.merge(munich_merged, munich_sum, on = 'borough')
munich_merged.sort_values(by=['amount of restaurants'], ascending=False, inplace=True)
munich_merged.head(5)

,postalcode,borough,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Sum,amount of restaurants
0,80331,Zentrum Marienplatz,48.137187,11.575501,7,German Restaurant,Bavarian Restaurant,Pizza Place,Italian Restaurant,Restaurant,24,24
1,81241,Pasing,48.147785,11.460701,7,Italian Restaurant,Vietnamese Restaurant,Fast Food Restaurant,German Restaurant,Theme Restaurant,18,18
2,80335,Hauptbahnhof,48.140458,11.557766,7,Middle Eastern Restaurant,German Restaurant,Bavarian Restaurant,Eastern European Restaurant,Italian Restaurant,11,11
3,85748,Garching,48.251388,11.650966,7,German Restaurant,Greek Restaurant,Italian Restaurant,Chinese Restaurant,Indian Restaurant,9,9
4,85748,Garching bei München,48.251388,11.650966,7,German Restaurant,Greek Restaurant,Italian Restaurant,Chinese Restaurant,Indian Restaurant,9,9


<a id='item4'></a>

## 4. Berlin: Data import and cleaning
## 4.1 libary import
If necessary, please run code line from '2. Munich'.

## 4.2 Berlin Data import with beautifulSoup

In [35]:
url='http://www.places-in-germany.com/14356-places-within-a-radius-of-15km-around-berlin.html'
req=requests.get(url)
soup=BeautifulSoup(req.text,"html.parser")
table = soup.find_all('table')
df=pd.read_html(str(table), header=0)[0]

In [36]:
df.head()

,Distance,Route,Postal code / Place,Population
0,1.2 km (0.8 miles),NaN,10115 Mitte,79582
1,2.1 km (1.3 miles),NaN,10119 Prenzlauer Berg,140881
2,2.8 km (1.7 miles),NaN,10115 Mitte,333534
3,2.9 km (1.8 miles),NaN,13347 Gesundbrunnen,82110
4,3.3 km (2.0 miles),NaN,10243 Friedrichshain-Kreuzberg,269398


delete double entry manually

In [37]:
df.drop(index=0, inplace=True)
df.shape

(97, 4)

In [38]:
df.rename({'Postal code / Place':'Borough'},axis=1, inplace=True)
berlin=df.Borough.str.split(" ",n=1,expand=True)
berlin.rename({0:'postalcode', 1:'borough'}, axis=1, inplace=True)
berlin

,postalcode,borough
1,10119,Prenzlauer Berg
2,10115,Mitte
3,13347,Gesundbrunnen
4,10243,Friedrichshain-Kreuzberg
5,10243,Friedrichshain
...,...,...
93,12459,Köpenick
94,12524,Altglienicke
95,16341,Schwanebeck bei Bernau bei Berlin
96,12305,Lichtenrade


test for NaN values

In [39]:
if berlin['borough'].isnull().sum() > 0:
    berlin.dropna(axis=0, inplace=True)
    print('NaN values deleted. New size ',berlin.shape)
else:
    print('no NaN values')

NaN values deleted. New size  (96, 2)


## 4.3 Folium Map Berlin

In [40]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

lat=[]
lon=[]

geolocator = Nominatim(user_agent='Cousera')

for line, borough in berlin.iterrows():
    
    try:
        adress= borough[0], ' Berlin ', borough[1]
        location = geolocator.geocode(adress)
        print(location)
        lat.append(location.latitude)
        lon.append(location.longitude)
    except:
        lat.append(np.nan)
        lon.append(np.nan)
    
berlin['latitude']=lat
berlin['longitude']=lon

Prenzlauer Berg, Winsviertel, Prenzlauer Berg, Pankow, Berlin, 10405, Deutschland
Mitte, Berlin, Deutschland
Gesundbrunnen, Mitte, Berlin, Deutschland
Friedrichshain-Kreuzberg, Berlin, Deutschland
Friedrichshain, Friedrichshain-Kreuzberg, Berlin, Deutschland
Tiergarten, Mitte, Berlin, Deutschland
Wedding, Mitte, Berlin, Deutschland
Moabit, Mitte, Berlin, Deutschland
U Platz der Luftbrücke, Mehringdamm, Kreuzberg, Friedrichshain-Kreuzberg, Berlin, 10965, Deutschland
Hansaviertel, Mitte, Berlin, Deutschland
Fennpfuhl, Lichtenberg, Berlin, 10369, Deutschland
Alt-Treptow, Treptow-Köpenick, Berlin, Deutschland
Weißensee, Pankow, Berlin, Deutschland
Pankow, Berlin, Deutschland
Heinersdorf, Romain-Rolland-Straße, Heinersdorf, Pankow, Berlin, 13089, Deutschland
Neukölln, Berlin, Deutschland
Lichtenberg, Berlin, Deutschland
Schöneberg, Tempelhof-Schöneberg, Berlin, Deutschland
Lichtenberg, Berlin, Deutschland
Rummelsburg, Lichtenberg, Berlin, Deutschland
Niederschönhausen, Pankow, Berlin, 13156

In [41]:
berlin.dropna(axis=0, inplace=True)
berlin.shape

(90, 4)

In [42]:
address = 'berlin'

geolocator = Nominatim(user_agent="Cousera")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('geo-coordinates of Berlin {}, {}.'.format(latitude, longitude))



# Creating Folium Map
map_berlin = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough in zip(berlin['latitude'], berlin['longitude'], berlin['borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_berlin)  
    
map_berlin

geo-coordinates of Berlin 52.5170365, 13.3888599.


## 4.4 Explore Venues with 4square

In [43]:
CLIENT_ID = 'MIXA2M5FTBG1IILFL5ATNOBSNUP4BU4J1BUOGIM03E1UJOBE' # your Foursquare ID
CLIENT_SECRET = 'ETQCTK31ZO4I0FVKYEEEHG2M0JGL0G4DCLQ1R1ECOOPHX33D' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
radius = 500
print("ID loaded")

ID loaded


In [44]:
berlin_venues = getNearbyVenues(names=berlin['borough'],
                                   latitudes=berlin['latitude'],
                                   longitudes=berlin['longitude']
                                  )

Prenzlauer Berg
Mitte
Gesundbrunnen
Friedrichshain-Kreuzberg
Friedrichshain
Tiergarten
Wedding
Moabit
Kreuzberg
Hansaviertel
Fennpfuhl
Alt-Treptow
Weißensee
Pankow
Heinersdorf
Neukölln
Lichtenberg
Schöneberg
Lichtenberg
Rummelsburg
Niederschönhausen
Tempelhof
Stadtrandsiedlung Malchow
Reinickendorf
Plänterwald
Charlottenburg
Wilhelmsruh
Alt-Hohenschönhausen
Wilmersdorf
Charlottenburg-Nord
Friedenau
Neu-Hohenschönhausen
Charlottenburg-Wilmersdorf
Friedrichsfelde
Malchow
Rosenthal
Baumschulenweg
Tempelhof-Schöneberg
Halensee
Blankenburg
Grunewald
Märkisches Viertel
Britz
Steglitz
Westend
Karlshorst
Wartenberg
Schmargendorf
Wittenau
Siemensstadt
Französisch Buchholz
Marzahn
Mariendorf
Biesdorf
Falkenberg
Tegel
Niederschöneweide
Lübars
Oberschöneweide
Waidmannslust
Lankwitz
Blankenfelde
Dahlem
Karow
Johannisthal
Haselhorst
Gropiusstadt
Buckow
Lichterfelde
Hermsdorf
Kaulsdorf
Marzahn-Hellersdorf
Schildow
Marienfelde
Hellersdorf
Ahrensfelde
Glienicke / Nordbahn
Rudow
Adlershof
Buch
Konradshö

In [45]:
print('4square provided', berlin_venues.shape , 'venues')
berlin_venues.head(5)

4square provided (1483, 7) venues


,borough,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Prenzlauer Berg,52.528634,13.420105,Bötzow Brauerei,52.530242,13.417293,Historic Site
1,Prenzlauer Berg,52.528634,13.420105,Soho House Cinema,52.527359,13.415775,Movie Theater
2,Prenzlauer Berg,52.528634,13.420105,Cowshed Active,52.527378,13.415791,Gym / Fitness Center
3,Prenzlauer Berg,52.528634,13.420105,The Store x Soho House Berlin,52.527525,13.415868,Boutique
4,Prenzlauer Berg,52.528634,13.420105,Rooftop Soho House,52.527533,13.415599,Roof Deck


In [46]:
print('There are {} uniques categories.'.format(len(berlin_venues['Venue Category'].unique())))
berlin_venues.groupby('Venue Category').Venue.count()

There are 245 uniques categories.


Venue Category
ATM                       2
Adult Boutique            1
African Restaurant        1
American Restaurant       2
Argentinian Restaurant    2
                         ..
Windmill                  1
Wine Bar                  2
Wine Shop                 5
Yoga Studio               2
Zoo Exhibit               2
Name: Venue, Length: 245, dtype: int64

<a id='item5'></a>

## 5. Analyze Boroughs of Berlin

In [47]:
berlin_restaurants = berlin_venues[berlin_venues['Venue Category'].str.contains('|'.join(restaurants))]

In [48]:
berlin_restaurants.head()

,borough,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
10,Prenzlauer Berg,52.528634,13.420105,The Store Kitchen,52.527506,13.415744,Bistro
11,Prenzlauer Berg,52.528634,13.420105,La Soupe Populaire,52.530595,13.416777,Modern European Restaurant
12,Prenzlauer Berg,52.528634,13.420105,Mondo Sardo,52.531076,13.422782,Italian Restaurant
19,Prenzlauer Berg,52.528634,13.420105,Leibhaftig,52.531392,13.416966,German Restaurant
20,Prenzlauer Berg,52.528634,13.420105,MontRaw Restaurant,52.531860,13.415882,Israeli Restaurant


## 5.1 Frequency of occurance

In [49]:
# one hot encoding
berlin_onehot = pd.get_dummies(berlin_restaurants[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
berlin_onehot['borough'] = berlin_restaurants['borough'] 

# move neighborhood column to the first column
fixed_columns = [berlin_onehot.columns[-1]] + list(berlin_onehot.columns[:-1])
berlin_onehot = berlin_onehot[fixed_columns]

berlin_onehot.head()

,borough,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Austrian Restaurant,Bavarian Restaurant,Beer Garden,Bistro,Brazilian Restaurant,...,Silesian Restaurant,Spanish Restaurant,Steakhouse,Sushi Restaurant,Syrian Restaurant,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
10,Prenzlauer Berg,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
11,Prenzlauer Berg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,Prenzlauer Berg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,Prenzlauer Berg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20,Prenzlauer Berg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### group rows by neighborhood and mean frequency of occurence of each category

In [50]:
print('Size of One Hot is ', berlin_onehot.shape, '.')
berlin_grouped = berlin_onehot.groupby('borough').mean().reset_index()
berlin_grouped.head()

Size of One Hot is  (392, 53) .


,borough,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Austrian Restaurant,Bavarian Restaurant,Beer Garden,Bistro,Brazilian Restaurant,...,Silesian Restaurant,Spanish Restaurant,Steakhouse,Sushi Restaurant,Syrian Restaurant,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Adlershof,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.25,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1,Alt-Hohenschönhausen,0.0,0.0,0.0,0.25,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
2,Alt-Treptow,0.0,0.0,0.0,0.00,0.0,0.0,0.142857,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.142857,0.0,0.0,0.0,0.142857
3,Blankenburg,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
4,Britz,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000


In [51]:
print('confirm the new size:', berlin_grouped.shape)

confirm the new size: (70, 53)


#### Let's print each neighborhood along with the top 5 most common venues

In [52]:
num_top_venues = 5

for hood in berlin_grouped['borough']:
    print("----"+hood+"----")
    temp = berlin_grouped[berlin_grouped['borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adlershof----
                venue  freq
0  Italian Restaurant  0.25
1    Greek Restaurant  0.25
2          Steakhouse  0.25
3         Pizza Place  0.25
4          Restaurant  0.00


----Alt-Hohenschönhausen----
                venue  freq
0    Doner Restaurant  0.25
1    Asian Restaurant  0.25
2   Indian Restaurant  0.25
3    Greek Restaurant  0.25
4  African Restaurant  0.00


----Alt-Treptow----
                venue  freq
0  Italian Restaurant  0.14
1         Beer Garden  0.14
2    Tapas Restaurant  0.14
3  Seafood Restaurant  0.14
4  Mexican Restaurant  0.14


----Blankenburg----
                      venue  freq
0          Greek Restaurant   1.0
1        African Restaurant   0.0
2       Japanese Restaurant   0.0
3       Lebanese Restaurant   0.0
4  Mediterranean Restaurant   0.0


----Britz----
                      venue  freq
0         German Restaurant   1.0
1        African Restaurant   0.0
2       Japanese Restaurant   0.0
3       Lebanese Restaurant   0.0
4  Mediterran

#### Let's put that into a _pandas_ dataframe

create the new dataframe and display top 5 venues

In [53]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
berlin_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
berlin_neighborhoods_venues_sorted['borough'] = berlin_grouped['borough']

for ind in np.arange(berlin_grouped.shape[0]):
    berlin_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(berlin_grouped.iloc[ind, :], num_top_venues)


#pd.set_option('display.max_rows', None)
berlin_neighborhoods_venues_sorted.head()

,borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Adlershof,Greek Restaurant,Italian Restaurant,Steakhouse,Pizza Place,Chinese Restaurant
1,Alt-Hohenschönhausen,Indian Restaurant,Greek Restaurant,Asian Restaurant,Doner Restaurant,Comfort Food Restaurant
2,Alt-Treptow,Vietnamese Restaurant,Italian Restaurant,Tapas Restaurant,Beer Garden,Seafood Restaurant
3,Blankenburg,Greek Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Halal Restaurant,German Restaurant
4,Britz,German Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Halal Restaurant,Greek Restaurant


## 5.2 Cluster Borough with k-means

In [54]:
# set number of clusters
kclusters = 10

berlin_grouped_clustering = berlin_grouped.drop('borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(berlin_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50] 

array([0, 0, 0, 9, 4, 2, 6, 0, 2, 4, 2, 8, 0, 1, 0, 0, 0, 0, 1, 3, 0, 0,
       8, 9, 0, 0, 0, 1, 2, 0, 3, 3, 5, 2, 7, 2, 4, 0, 0, 3, 5, 4, 0, 2,
       0, 0, 2, 0, 0, 0], dtype=int32)

create dataframe 'berlin_merged'

In [55]:
# add clustering labels
berlin_neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [56]:
berlin_merged = berlin

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
berlin_merged = berlin_merged.merge(berlin_neighborhoods_venues_sorted.set_index('borough'), on='borough')

#pd.set_option('display.max_rows', None)
berlin_merged

,postalcode,borough,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,10119,Prenzlauer Berg,52.528634,13.420105,0,German Restaurant,Israeli Restaurant,Italian Restaurant,Thai Restaurant,Bistro
1,10115,Mitte,52.517885,13.404060,4,German Restaurant,Vietnamese Restaurant,Spanish Restaurant,Italian Restaurant,Restaurant
2,13347,Gesundbrunnen,52.550920,13.384846,0,Turkish Restaurant,Italian Restaurant,Halal Restaurant,Syrian Restaurant,Falafel Restaurant
3,10243,Friedrichshain-Kreuzberg,52.501115,13.444285,0,Vietnamese Restaurant,Pizza Place,Indian Restaurant,Falafel Restaurant,Asian Restaurant
4,10243,Friedrichshain,52.512215,13.450290,0,Middle Eastern Restaurant,Doner Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Italian Restaurant
...,...,...,...,...,...,...,...,...,...,...
65,13581,Spandau,52.535788,13.197792,0,Restaurant,Italian Restaurant,Asian Restaurant,Bistro,Fast Food Restaurant
66,12623,Mahlsdorf,52.508699,13.613162,2,Italian Restaurant,Greek Restaurant,Indian Restaurant,Halal Restaurant,German Restaurant
67,12459,Köpenick,52.453910,13.576413,3,Indian Restaurant,German Restaurant,Burger Joint,Comfort Food Restaurant,Halal Restaurant
68,12305,Lichtenrade,52.393456,13.402040,5,Doner Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Halal Restaurant,Greek Restaurant


## 5.3 Cluster Visualization

In [57]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(berlin_merged['latitude'], berlin_merged['longitude'], berlin_merged['borough'], berlin_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5.4 evaluate target criteria according to 'berlin_restaurants'

In [58]:
berlin_sum = pd.get_dummies(berlin_restaurants[['Venue Category']], prefix="", prefix_sep="")

berlin_sum['borough'] = berlin_restaurants['borough']

#Fix Column Order
#fixed_columns = [berlin_sum.columns[-1]] + list(berlin_sum.columns[:-1])
#berlin_sum = berlin_sum[fixed_columns]
berlin_sum

,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Austrian Restaurant,Bavarian Restaurant,Beer Garden,Bistro,Brazilian Restaurant,Burger Joint,...,Spanish Restaurant,Steakhouse,Sushi Restaurant,Syrian Restaurant,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,borough
10,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Prenzlauer Berg
11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Prenzlauer Berg
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Prenzlauer Berg
19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Prenzlauer Berg
20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Prenzlauer Berg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Köpenick
1476,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Lichtenrade
1477,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Steglitz-Zehlendorf
1478,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Steglitz-Zehlendorf


Sum amount of each venues out of target definition 'berlin_restaurants'

In [59]:
berlin_sum['amount of restaurants']=berlin_sum.sum(axis=1)
berlin_sum = berlin_sum.groupby('borough').sum().reset_index()
berlin_sum

,borough,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Austrian Restaurant,Bavarian Restaurant,Beer Garden,Bistro,Brazilian Restaurant,...,Spanish Restaurant,Steakhouse,Sushi Restaurant,Syrian Restaurant,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,amount of restaurants
0,Adlershof,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,4
1,Alt-Hohenschönhausen,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
2,Alt-Treptow,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,7
3,Blankenburg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,Britz,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,Westend,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
66,Wilhelmsruh,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
67,Wilmersdorf,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,1,0,0,2,20
68,Wittenau,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5


keep only sum as criteria and add it to overview 'berlin_merged'<br>
show only top5 boroughs according to amount of restaurants

In [60]:
berlin_sum = berlin_sum[['borough','amount of restaurants']]

berlin_merged = pd.merge(berlin_merged, berlin_sum, on = 'borough')
berlin_merged.sort_values(by=['amount of restaurants'], ascending=False, inplace=True)
berlin_merged.head(5)

,postalcode,borough,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,amount of restaurants
3,10243,Friedrichshain-Kreuzberg,52.501115,13.444285,0,Vietnamese Restaurant,Pizza Place,Indian Restaurant,Falafel Restaurant,Asian Restaurant,29
14,12043,Neukölln,52.481150,13.435350,0,Italian Restaurant,Middle Eastern Restaurant,Bistro,Vegetarian / Vegan Restaurant,Lebanese Restaurant,22
23,10707,Wilmersdorf,52.487115,13.320330,0,Vietnamese Restaurant,Doner Restaurant,French Restaurant,Italian Restaurant,Burger Joint,20
6,10551,Moabit,52.530102,13.342542,0,German Restaurant,Doner Restaurant,Burger Joint,Vegetarian / Vegan Restaurant,Italian Restaurant,20
15,10777,Schöneberg,52.482157,13.355190,0,Vietnamese Restaurant,Restaurant,Doner Restaurant,Italian Restaurant,Argentinian Restaurant,19


<a id='item6'></a>

## 6. result and discussion

Now we've got a top5 list of each City with amount and most frequent kind of restaurantof restaurants:  

### 6.1 Berlin

In [61]:
berlin_merged.head(5)

,postalcode,borough,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,amount of restaurants
3,10243,Friedrichshain-Kreuzberg,52.501115,13.444285,0,Vietnamese Restaurant,Pizza Place,Indian Restaurant,Falafel Restaurant,Asian Restaurant,29
14,12043,Neukölln,52.481150,13.435350,0,Italian Restaurant,Middle Eastern Restaurant,Bistro,Vegetarian / Vegan Restaurant,Lebanese Restaurant,22
23,10707,Wilmersdorf,52.487115,13.320330,0,Vietnamese Restaurant,Doner Restaurant,French Restaurant,Italian Restaurant,Burger Joint,20
6,10551,Moabit,52.530102,13.342542,0,German Restaurant,Doner Restaurant,Burger Joint,Vegetarian / Vegan Restaurant,Italian Restaurant,20
15,10777,Schöneberg,52.482157,13.355190,0,Vietnamese Restaurant,Restaurant,Doner Restaurant,Italian Restaurant,Argentinian Restaurant,19


### 6.2 Munich

In [85]:
munich_merged.head(5)

,postalcode,borough,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Sum,amount of restaurants
0,80331,Zentrum Marienplatz,48.137187,11.575501,7,German Restaurant,Bavarian Restaurant,Pizza Place,Italian Restaurant,Restaurant,24,24
1,81241,Pasing,48.147785,11.460701,7,Italian Restaurant,Vietnamese Restaurant,Fast Food Restaurant,German Restaurant,Theme Restaurant,18,18
2,80335,Hauptbahnhof,48.140458,11.557766,7,Middle Eastern Restaurant,German Restaurant,Bavarian Restaurant,Eastern European Restaurant,Italian Restaurant,11,11
3,85748,Garching,48.251388,11.650966,7,German Restaurant,Greek Restaurant,Italian Restaurant,Chinese Restaurant,Indian Restaurant,9,9
4,85748,Garching bei München,48.251388,11.650966,7,German Restaurant,Greek Restaurant,Italian Restaurant,Chinese Restaurant,Indian Restaurant,9,9


`Berlin - Friedrichshain-Kreuzberg` has the highest number of restaurants of `29`. <br>
But `Munich - Marienplatz` seems also to be a good choice with `24` different restaurants. <br>
So she will decide based on most common restaurants.

## 6.3 Explore Trending Venues

Since it's possible to get at least _one_ premium request, we use __*trending*__ API. <br>
Most probably she will pick `Friedrichshain-Kreuzberg` with the highest number of restaurants of `29`. <br>
As soon as we arrive, we will look for the places with the highest foot traffic. So let's do that and get the trending venues around.

In [78]:
latitude = berlin_merged._get_value(3, 'latitude')
longitude = berlin_merged._get_value(3, 'longitude')
print("borough =", berlin_merged._get_value(3, 'borough'), ", latitude =", latitude, ", longitude =", longitude)

borough = Friedrichshain-Kreuzberg , latitude = 52.5011154 , longitude = 13.4442848


In [79]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ffd6f19aca2493433cd08d3'},
 'response': {'venues': []}}

### Check if any venues are trending at this time


In [80]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [81]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

Now, depending on when you run the above code, you might get different venues since the venues with the highest foot traffic are fetched live. 


### Visualize trending venues


In [ ]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [ ]:
# display map
venues_map

[back to top](#top)